# Movielens 영화 SBR
-----
### 프로젝트 - 

**`Movielens 1M Dataset`**을 기반으로, Session based Recommendation 시스템을 제작해 보겠습니다.

## 의

>* 의료영상 이미지는 **개인정보 보호 등**의 이슈로 인해 데이터를 구하는 것이 어려움
>* 특히 희소병을 다루는 경우에는 데이터를 입수하는 것 자체가 드문 일
>* 라벨링 작업 자체가 전문적 지식을 요구하므로 date set 구축 비용이 상승
>* 음성/양성 데이터 간 imbalance가 심하기 때문에 학습에 주의가 필요
>* 이미지만으로 진단이 쉽지 않아 다른 데이터와 결합해서 해석해야 할 수도 있음

최근의 발달한 딥러닝 기술은 숙련자 수준 이상의 정확도를 바탕으로 **영상분석 인력의 개인적 편차, 주관적 판단, 피로에 의한 오진 등의 부정확성을 극복**할 수 있는 좋은 대안으로 인정받고 있어요.<br>의료영상 처리를 위해서는 딥러닝 영상처리 기술뿐만 아니라, 의료 도메인 지식 및 의료영상에 대한 명확한 이해가 아울러 필요합니다.

### 의료 영상 처리의 핵심 영역들
---
의료 영상 처리는 각 과정마다 다양한 개념과 기법들이 사용됩니다.<br>이런 과정은 크게 세 단계로 구분할 수 있는데요, 이미지 생성, 이미지 컴퓨팅, 이미지 관리로 나눌 수 있습니다.
![](https://cdn.epnc.co.kr/news/photo/201907/91092_81350_2153.jpg)
---
영상 방식에 관계없이, 데이터 수집 프로세스는 물리적 인자들을 검출하고 수집한 신호를 전처리한 후 디지털화하는 과정을 거칩니다.<br>아래 그림은 대부분의 주요 의료 영상 방식에 적용되는 과정을 보여줍니다.
![](https://cdn.epnc.co.kr/news/photo/201907/91092_81351_2316.jpg)
---
![](https://cdn.epnc.co.kr/news/photo/201907/91092_81352_2514.jpg)
위의 표는 이미지 컴퓨팅과 이미지 관리 단계의 과제와 개발 동향을 요약한 것입니다.<br>

출처 : 테크월드뉴스(http://www.epnc.co.kr)

## 순서
> **Step1.** 데이터의 전처리<br>
**Step2.** 미니 배치의 구성<br>
**Step3.** 모델 구성<br>
**Step4.** 모델 학습<br>
**Step5.** 모델 테스트<br>

## 용어정리
---

**✓ Session-Based Recommendation**이란?
>* 말 그대로 세션 데이터를 기반으로 **유저가 다음에 클릭 또는 구매할 아이템을 예측하는 추천**

>* **Session이란**이란?
>> * 유저가 서비스를 이용하면서 발생하는 중요한 정보를 담은 데이터를 말하며, 서버 쪽에 저장
>> * 유저의 행동 데이터들은 유저측 브라우저를 통해 쿠키의 형태로 저장
>> * 쿠키는 세션과 상호작용하면서 정보를 주고 받음
>> * 이번 자료에서는 세션을 **브라우저가 종료되기 전까지 유저의 행동을 담은 시퀀스 데이터**와 같은 의미로 사용
>* 참고자료
>>* [쿠키, 세션, 캐시가 뭔가요?](https://www.youtube.com/watch?v=OpoVuwxGRDI)
>>* [쿠키, 세션이란?](https://chrisjune-13837.medium.com/web-%EC%BF%A0%ED%82%A4-%EC%84%B8%EC%85%98%EC%9D%B4%EB%9E%80-aa6bcb327582)


**✓ 데이터 로드**

1) wget으로 데이터 다운로드
> $ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

2) 다운받은 데이터를 작업디렉토리로 이동
> $ mv ml-1m.zip ~/aiffel/yoochoose-data

3) 압축 해제
> $ cd ~/aiffel/yoochoose-data && unzip ml-1m.zip

In [ ]:
data_path = Path(os.getenv('HOME')+'/aiffel/yoochoose-data/ml-1m') 
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    return data

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data

- 여기서 이전 실습내역과 가장 크게 다른 부분은 바로 SessionID 대신 UserID 항목이 들어갔다는 점입니다. 이 데이터셋은 명확한 1회 세션의 SessionID를 포함하지 않고 있습니다. 그래서 이번에는 UserID가 SessionID 역할을 해야 합니다.

- Rating 정보가 포함되어 있습니다. 이전 실습내역에서는 이런 항목이 포함되어 있지 않았으므로, 무시하고 제외할 수 있습니다. 하지만, 직전에 봤던 영화가 맘에 들었는지 여부가 비슷한 영화를 더 고르게 하는 것과 상관이 있을 수도 있습니다. 아울러, Rating이 낮은 데이터를 어떻게 처리할지도 고민해야 합니다.

- Time 항목에는 UTC time 가 포함되어, 1970년 1월 1일부터 경과된 초단위 시간이 기재되어 있습니다.

### Step1. 데이터의 전처리
---
위와 같이 간단히 구성해 본 데이터셋을 꼼꼼이 살펴보면서 항목별 기본분석, session length, session time, cleaning 등의 작업을 진행합니다.
특히, 이 데이터셋에서는 Session이 아닌 UserID 단위로 데이터가 생성되어 있으므로, 이를 Session 단위로 어떻게 해석할지에 주의합니다.

오늘 사용할 패키지를 불러옵니다.<br>NTLK와 NLTK 데이터셋이 설치되어있지 않은 환경이라면 우선 NLTK를 설치하고 NTLK의 데이터셋을 다운로드해주세요.<br>터미널에서 **`pip install [패키지명]`**을 사용해 필요한 패키지를 설치해 주세요.<br>아래 명령어를 복사해서 설치해도 됩니다.

> $ pip install nltk

**✓ 패키지(라이브러리)들을 import 해보겠습니다.**

> $ pip install beautifulsoup4

nltk 패키지에서 불용어 사전을 다운로드 받고, 데이터 전처리를 위한 나머지 패키지도 함께 불러옵니다.

### Step2. 미니 배치의 구성
---
실습코드 내역을 참고하여 데이터셋과 미니 배치를 구성해 봅시다. Session-Parallel Mini-Batch의 개념에 따라, 학습 속도의 저하가 최소화될 수 있도록 구성합니다.
단, 위 Step 1에서 Session 단위를 어떻게 정의했느냐에 따라서 Session-Parallel Mini-Batch이 굳이 필요하지 않을 수도 있습니다.

**✓ 예를 들어 좌우반전 정도의 augmentation만 도입한다고 한다면 다음과 같이 진행될 것입니다.**

**✓ 텍스트 정규화와 불용어 제거**

### Step3. 모델 구성
---
이 부분도 실습코드 내역을 참고하여 다양하게 모델 구조를 시도해볼 수 있습니다.

### Step4. 모델 학습
---
다양한 하이퍼파라미터를 변경해 보며 검증해 보도록 합니다. 실습코드에 언급되었던 Recall, MRR 등의 개념들도 함께 관리될 수 있도록 합니다.

### Step5. 모델 테스트
---
미리 구성한 테스트셋을 바탕으로 Recall, MRR 을 확인해 봅니다.

# 루브릭 평가 기준
---
번호|평가문항|상세기준
---|---|---
1 |Movielens 데이터셋을 session based recommendation 관점으로 전처리하는 과정이 체계적으로 진행되었다.|데이터셋의 면밀한 분석을 토대로 세션단위 정의 과정(길이분석, 시간분석)을 합리적으로 수행한 과정이 기술되었다.
2 |RNN 기반의 예측 모델이 정상적으로 구성되어 안정적으로 훈련이 진행되었다.|적절한 epoch만큼의 학습이 진행되는 과정에서 train loss가 안정적으로 감소하고, validation 단계에서의 Recall, MRR이 개선되는 것이 확인된다.
3 |세션정의, 모델구조, 하이퍼파라미터 등을 변경해서 실험하여 Recall, MRR 등의 변화추이를 관찰하였다.|3가지 이상의 변화를 시도하고 그 실험결과를 체계적으로 분석하였다.

# 회고
---